In [1]:
import numpy as np
import pandas as pd

from sklearn.cross_validation import train_test_split # to divide train and test set
from sklearn import preprocessing # for feature scaling

# feature selection
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import f_regression

# import linear model
from sklearn.linear_model import LinearRegression

# model evaluation
from sklearn import cross_validation

In [2]:
cd Dropbox/Portfolio/DataScience-Portfolio/KDD-1998

/Users/Capgemini/Dropbox/Portfolio/DataScience-Portfolio/KDD-1998


### Load data and separate in Classifier and Labels

In [3]:
# load data
kdd = pd.read_csv('data_reg2.csv')

# generate X and Y for preditions
Y = np.ravel(kdd.TARGET_D)  # to flatten array
X = kdd.drop('TARGET_D', axis = 1)

In [4]:
X.shape

(4829, 1913)

### Functions for feature scaling

In [5]:
# Feature scaling - normalisation
def standarisation(train, test):
    scaler = preprocessing.StandardScaler().fit(train)
    train = scaler.transform(train)
    test = scaler.transform(test)
    return train, test

# Feature scaling - MinMax Scaler (scales between 0 and 1)
def minMax_standarisation(train, test):
    scaler = preprocessing.MinMaxScaler().fit(train)
    train = scaler.transform(train)
    test = scaler.transform(test)
    return train, test

### Function for feature selection

In [6]:
# feature selection function
def feat_select(model, xtrain, test, ytrain):
    selector = model
    selector.fit(xtrain, ytrain)
    X_train_new = selector.transform(xtrain)
    X_test_new = selector.transform(test)
    return X_train_new, X_test_new

### Function for Linear Regression

In [7]:
# Create linear regression object
def linReg_mod(Xtrain, Ytrain, Xtest, Ytest):
    regr = LinearRegression()

    # Train the model using the training sets
    regr.fit(Xtrain, Ytrain)
    
    # Measures for training set
    print("Train set")
    print("Residual sum of squares: %.2f" % np.mean((regr.predict(Xtrain) - Ytrain) ** 2))
    # Explained variance score: 1 is perfect prediction
    print('Rsquared: %.2f' % regr.score(Xtrain, Ytrain))
    print('=================')
    # Measures for testing set
    print("Test set")
    print("Residual sum of squares: %.2f" % np.mean((regr.predict(Xtest) - Ytest) ** 2))
    # Explained variance score: 1 is perfect prediction
    print('Rsquared: %.2f' % regr.score(Xtest, Ytest))
    return regr

### Function to print results of model

In [8]:
# Print results of model
def print_results(clf, X_train, Y_train, X_test, Y_test):
    print("Train set")
    print("Residual sum of squares: %.2f" % np.mean((clf.predict(X_train) - Y_train) ** 2))
    print('Rsquared: %.2f' % clf.score(X_train, Y_train))
    print('=================')
    print("Test set")
    print("Residual sum of squares: %.2f" % np.mean((clf.predict(X_test) - Y_test) ** 2))
    print('Rsquared: %.2f' % clf.score(X_test, Y_test))

### Function to split train and test set and normalise predictors

In [9]:
# function to split test and train and normalise
def split_standarise(X,Y):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=42)
    X_train, X_test = standarisation(X_train, X_test)
    return X_train, X_test, Y_train, Y_test

## First linear model

In [10]:
# separate testing and training set + normalise
X_train, X_test, Y_train, Y_test = split_standarise(X,Y)
X_train.shape, X_test.shape

((2897, 1913), (1932, 1913))

In [11]:
# select top 10% features
X_train, X_test = feat_select(SelectPercentile(f_regression, percentile = 10), X_train, X_test, Y_train)
X_train.shape, X_test.shape

((2897, 192), (1932, 192))

In [12]:
# run first linear model with 192 features
regr = linReg_mod(X_train, Y_train, X_test, Y_test)

Train set
Residual sum of squares: 51.26
Rsquared: 0.68
Test set
Residual sum of squares: 1048359703987778702278656.00
Rsquared: -6988277458269901946880.00


Overfitting to the training set.

### Second linear model

In [13]:
# separate train and test and normalise
X_train, X_test, Y_train, Y_test = split_standarise(X,Y)

# select 10 best features
X_train, X_test = feat_select(SelectKBest(f_regression, k=10), X_train, X_test, Y_train)
X_train.shape

(2897, 10)

In [14]:
# run second linear model with 10 features
regr2 = linReg_mod(X_train, Y_train, X_test, Y_test)

Train set
Residual sum of squares: 73.53
Rsquared: 0.54
Test set
Residual sum of squares: 84.51
Rsquared: 0.44


Better model. Although the R squared is quite low.

### Third linear Model

In [15]:
# sep train and test, normalise, select top 10% features
X_train, X_test, Y_train, Y_test = split_standarise(X,Y)
X_train, X_test = feat_select(SelectPercentile(f_regression, percentile = 10), X_train, X_test, Y_train)

In [16]:
# Recursive Feature Selection (RFS)
regr_RFS = SelectFromModel(regr, prefit=True)
X_train = regr_RFS.transform(X_train)
X_test = regr_RFS.transform(X_test)

X_train.shape

(2897, 28)

In [17]:
regr3 = linReg_mod(X_train, Y_train, X_test, Y_test)

Train set
Residual sum of squares: 91.55
Rsquared: 0.42
Test set
Residual sum of squares: 4944838232850513219106635776.00
Rsquared: -32961875037714399994314752.00


Overfits.

### Try different feature scaling (MinMax)

In [18]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=42)

# normalise features with MinMax
X_train, X_test = minMax_standarisation(X_train, X_test)

# select 10 best features
X_train, X_test = feat_select(SelectKBest(f_regression, k=10), X_train, X_test, Y_train)
regr3 = linReg_mod(X_train, Y_train, X_test, Y_test)

Train set
Residual sum of squares: 73.53
Rsquared: 0.54
Test set
Residual sum of squares: 84.51
Rsquared: 0.44


No improvement with respect to first model with 10 best features

## Try normalizing label with log 

In [19]:
regr3 = linReg_mod(X_train, np.log(Y_train), X_test, np.log(Y_test))

Train set
Residual sum of squares: 0.19
Rsquared: 0.56
Test set
Residual sum of squares: 0.19
Rsquared: 0.51


A better is obtained if we estimate the log of the amount donated.

### Feature selection with grid search and BayesianRidge

In [20]:
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import KFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import BayesianRidge

# split and normalise
X_train, X_test, Y_train, Y_test = split_standarise(X,Y)

ridge = BayesianRidge()      # model object
cv = KFold(len(Y_train), 2)  # cross-validation generator for model selection
anova = SelectPercentile(f_regression) # feature selection

clf = Pipeline([('anova', anova), ('ridge', ridge)])

# Select the optimal percentage of features with grid search
clf = GridSearchCV(clf, {'anova__percentile': [2, 5, 10, 20, 30, 50]}, cv=cv)
clf.fit(X_train, Y_train)  # set the best parameters

GridSearchCV(cv=sklearn.cross_validation.KFold(n=2897, n_folds=2, shuffle=False, random_state=None),
       error_score='raise',
       estimator=Pipeline(steps=[('anova', SelectPercentile(percentile=10,
         score_func=<function f_regression at 0x10fb89048>)), ('ridge', BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
       fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
       normalize=False, tol=0.001, verbose=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'anova__percentile': [2, 5, 10, 20, 30, 50]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [21]:
clf.best_params_

{'anova__percentile': 2}

In [22]:
print_results(clf, X_train, Y_train, X_test, Y_test)

Train set
Residual sum of squares: 65.28
Rsquared: 0.59
Test set
Residual sum of squares: 78.39
Rsquared: 0.48


This model is a little bit better than the linear regression with 10 features.

### Same as above but with feature agglomeration

In [23]:
from sklearn.cluster import FeatureAgglomeration

X_train, X_test, Y_train, Y_test = split_standarise(X,Y)

ridge = BayesianRidge() # model object

# Ward agglomeration followed by BayesianRidge
ward = FeatureAgglomeration(n_clusters=10)
clf = Pipeline([('ward', ward), ('ridge', ridge)])
# Select the optimal number of parcels with grid search
clf = GridSearchCV(clf, {'ward__n_clusters': [10, 20, 30]}, n_jobs=1, cv=cv)
clf.fit(X_train, Y_train)  # set the best parameters

GridSearchCV(cv=sklearn.cross_validation.KFold(n=2897, n_folds=2, shuffle=False, random_state=None),
       error_score='raise',
       estimator=Pipeline(steps=[('ward', FeatureAgglomeration(affinity='euclidean', compute_full_tree='auto',
           connectivity=None, linkage='ward', memory=Memory(cachedir=None),
           n_clusters=10, n_components=None,
           pooling_func=<function mean at 0x10bcad378>)), ('ridge', BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
       fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
       normalize=False, tol=0.001, verbose=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'ward__n_clusters': [10, 20, 30]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [24]:
clf.best_params_

{'ward__n_clusters': 30}

In [25]:
print_results(clf, X_train, Y_train, X_test, Y_test)

Train set
Residual sum of squares: 102.37
Rsquared: 0.35
Test set
Residual sum of squares: 102.09
Rsquared: 0.32


Somewhat worse model than the previous ones

### Support vector regression

In [26]:
from sklearn.svm import SVR

X_train, X_test, Y_train, Y_test = split_standarise(X,Y)
X_train, X_test = feat_select(SelectPercentile(f_regression, percentile = 5), X_train, X_test, Y_train)

# SVM regularization parameter
for C in [0.01, .1, 1.0, 10]:
    # SVC with a Linear Kernel 
    svr = SVR(kernel='linear', C=C).fit(X_train, Y_train)
    print_results(svr, X_train, Y_train, X_test, Y_test)
    print()
    print()

Train set
Residual sum of squares: 76.66
Rsquared: 0.52
Test set
Residual sum of squares: 79.74
Rsquared: 0.47


Train set
Residual sum of squares: 83.36
Rsquared: 0.47
Test set
Residual sum of squares: 75.70
Rsquared: 0.50


Train set
Residual sum of squares: 92.47
Rsquared: 0.42
Test set
Residual sum of squares: 79.10
Rsquared: 0.47


Train set
Residual sum of squares: 94.01
Rsquared: 0.41
Test set
Residual sum of squares: 79.27
Rsquared: 0.47




The support vector regression with a C parameter of 0.1 offers the best model at the minute, as it shows the highest R squared for the training set (0.5)

### Try same model with the logarithm of the target variable

In [28]:
svr = SVR(kernel='linear', C=0.1).fit(X_train, (np.log(Y_train)))
print_results(svr, X_train, np.log(Y_train), X_test, np.log(Y_test))

Train set
Residual sum of squares: 0.19
Rsquared: 0.56
Test set
Residual sum of squares: 0.18
Rsquared: 0.55


This model offers a 10% improvement with respect to the previous one. 

In [29]:
# Try Support vector regression with polynomial kernel
X_train, X_test, Y_train, Y_test = split_standarise(X,Y)
X_train, X_test = feat_select(SelectPercentile(f_regression, percentile = 5), X_train, X_test, Y_train)

# SVM regularization parameter
for C in [.1, 1.0, 10]:
    for degree in range(1,5):
        # SVR with a Poly Kernel
        svr = SVR(kernel='poly', degree = degree, C=C).fit(X_train, Y_train)
        print(C, degree)
        print_results(svr, X_train, Y_train, X_test, Y_test)
        print()
        print()

0.1 1
Train set
Residual sum of squares: 91.74
Rsquared: 0.42
Test set
Residual sum of squares: 93.25
Rsquared: 0.38


0.1 2
Train set
Residual sum of squares: 90.39
Rsquared: 0.43
Test set
Residual sum of squares: 105.47
Rsquared: 0.30


0.1 3
Train set
Residual sum of squares: 79.96
Rsquared: 0.49
Test set
Residual sum of squares: 106.63
Rsquared: 0.29


0.1 4
Train set
Residual sum of squares: 72.12
Rsquared: 0.54
Test set
Residual sum of squares: 119.36
Rsquared: 0.20


1.0 1
Train set
Residual sum of squares: 76.62
Rsquared: 0.52
Test set
Residual sum of squares: 79.72
Rsquared: 0.47


1.0 2
Train set
Residual sum of squares: 66.59
Rsquared: 0.58
Test set
Residual sum of squares: 93.24
Rsquared: 0.38


1.0 3
Train set
Residual sum of squares: 54.07
Rsquared: 0.66
Test set
Residual sum of squares: 114.09
Rsquared: 0.24


1.0 4
Train set
Residual sum of squares: 50.46
Rsquared: 0.68
Test set
Residual sum of squares: 155.41
Rsquared: -0.04


10 1
Train set
Residual sum of squares: 83

We observe that the polynomial kernel overfits to the training set.

### Dimensionality Reduction with PCA

In [30]:
from sklearn.decomposition import PCA

X_train, X_test, Y_train, Y_test = split_standarise(X,Y)
X_train, X_test = feat_select(SelectPercentile(f_regression, percentile = 5), X_train, X_test, Y_train)

pca = PCA(n_components=5, copy=True, whiten=False)
pca.fit(X_train)
print(pca.explained_variance_ratio_)

[ 0.11879738  0.08759205  0.07831258  0.07250979  0.06881569]


In [31]:
X_train = pca.transform(X_train)
X_train.shape

(2897, 5)

In [32]:
X_test = pca.transform(X_test)

In [33]:
regrPca = linReg_mod(X_train, Y_train, X_test, Y_test)

Train set
Residual sum of squares: 92.40
Rsquared: 0.42
Test set
Residual sum of squares: 95.16
Rsquared: 0.37


## Conclusion

The best model is the support vector regression with a C of 0.1, utilising the 5% best features selected by univariate feature selection. This model estimates the log(donation amount). The R squared is 0.55.

The second best model is the bayesianRidge utilising 2% best features selected by univariate feature selection. I haven't optimised the parameters of the Bayesian Ridge, and I have not attempted to estimate the log of the donation amount, which will most likely generate an improvement in the predictive power of the model, so there is definitely some room to improve this model.

The third model is a linear regression model utilising the 10 best features selected by univariate feature selection. I will explore this model further below, to identify the variables that are important contributors for the accurate prediction of the donation amount.

In [34]:
X_train, X_test, Y_train, Y_test = split_standarise(X,Y)

selector = SelectKBest(f_regression, k=10)
selector.fit(X_train, Y_train)
X_train_new = selector.transform(X_train)
X_test_new = selector.transform(X_test)

In [35]:
X_train_new.shape

(2897, 10)

In [36]:
pval = pd.Series(selector.pvalues_)
pval.sort_values().head(10)

301      0.000000e+00
1894    2.883861e-208
303     2.781228e-164
436     1.115719e-123
300     1.237474e-114
305      7.678342e-88
451      4.169689e-78
521      4.697240e-78
1891     2.228358e-74
1904     3.024232e-71
dtype: float64

In [37]:
ind = pval.sort_values().head(10).index
ind

Int64Index([301, 1894, 303, 436, 300, 305, 451, 521, 1891, 1904], dtype='int64')

In [38]:
# columns with the most significant p-values: these were used in the model
pd.Series(X.columns[ind])

0      LASTGIFT
1      RFA_2A_G
2       AVGGIFT
3     RFA_2_L1G
4      MAXRAMNT
5        RFA_2F
6     RFA_3_A1G
7     RFA_4_A1G
8      RFA_2A_D
9    MDMAUD_A_C
dtype: object

In [39]:
feat = pd.DataFrame(X.columns[ind], columns = ['feature'])
feat['p_value']= pd.DataFrame(np.array(pval.sort_values().head(10)))
feat

,feature,p_value
0,LASTGIFT,0.000000e+00
1,RFA_2A_G,2.883861e-208
2,AVGGIFT,2.781228e-164
3,RFA_2_L1G,1.115719e-123
4,MAXRAMNT,1.237474e-114
5,RFA_2F,7.678342e-88
6,RFA_3_A1G,4.169689e-78
7,RFA_4_A1G,4.697240e-78
8,RFA_2A_D,2.228358e-74
9,MDMAUD_A_C,3.024232e-71


The most important features for the prediction of the donation amount are the amount donated last (LASTGIFT), the average amount donated in general (AVGGIFT), and the maximum amount donated so far (MAXRAMNT) among other variables that indicate certain codes pertinent to the charity.